In [4]:
from sklearn.cluster import AgglomerativeClustering
from shapely.geometry import MultiPoint, Polygon, MultiPolygon

import pandas as pd
import datetime
from dateutil import parser
import numpy as np
import geopandas as gpd
from dateutil import parser


datafile=r'C:\Users\zhuj\Documents\PythonScripts\file0810back21days.csv'
data=pd.read_csv(datafile)
data.Date=[parser.parse(t).date() for t in data.Date]
start_date=None
if start_date is None:
        start_date=np.min(data.Date.values)
        recheader=['id','centroid_x','centroid_y']
end_date=datetime.datetime.now().date()
datevals=[start_date+datetime.timedelta(days=i) for i in range((end_date-start_date).days+1)]
recheader.extend(datevals)
gpframe=gpd.GeoDataFrame(data,geometry=gpd.points_from_xy(data.lon, data.lat))
gpframe.crs={'init': 'epsg:4326'}
gpframe=gpframe.to_crs({'init': 'epsg:26917'})
data['x']=[geom.x for geom in gpframe.geometry]
data['y']=[geom.y for geom in gpframe.geometry]
data=data.drop(columns=['geometry'])
xy=data[['x','y']].values
latlon=data[['lon','lat']].values
   
dates=data['Date'].values
minneighbors=10
maxdistance=1000
    #empty data frame for storing clusters
clusterData=pd.DataFrame(columns=recheader)
    #run agglomerative clustering algorithm
clustering = AgglomerativeClustering(n_clusters=None,linkage='single',distance_threshold=maxdistance,compute_full_tree=True).fit(xy)
    #cluster labels
labels = clustering.labels_
total_clusters = len(set(labels))
k=0
geometries=[]
geometries_line=[]
ids=[]
for i in range(total_clusters):
        ltln=latlon[labels==i]
        date=dates[labels==i]
        prjpoints=xy[labels==i]
        if len(ltln)!=0 and len(ltln)>=minneighbors:
            tdframe=pd.DataFrame(columns=['date'])
            tdframe.date=datevals
            newdframe=pd.DataFrame(columns=['date','counts'])
           
            unique, counts = np.unique(date, return_counts=True)
            newdframe.date=unique
            newdframe.counts=counts
            tdframe=tdframe.merge(newdframe,on='date',how='outer')
            tdframe=tdframe.fillna(0)
          
            cnts=np.cumsum(tdframe.counts.values)
          
            cnts[cnts<minneighbors]=0
            tdframe.counts=cnts
  
            geom=MultiPoint(ltln)
            dat=[k,geom.centroid.x,geom.centroid.y]
            ids.append(k)
            geom_proj=MultiPoint(prjpoints)
         
            geom_hull=geom_proj.convex_hull
            if geom_hull.geom_type!='Polygon':
                geom_hull=geom_hull.buffer(maxdistance)
            geom_line=geom_hull.boundary
            geometries.append(geom_hull)
            geometries_line.append(geom_line)
            dat.extend(tdframe.counts.values.tolist())
            clusterData.loc[len(clusterData)]=dat
            k+=1
  
            clusterData.to_csv(datafile.replace('.csv','')+"_centroids__"+str(maxdistance)+"_"+str(minneighbors)+'.csv',index=False)
  
            gdf = gpd.GeoDataFrame(pd.DataFrame(ids,columns=['id']), geometry=geometries,crs={'init': 'epsg:26917'})
            gdf_line = gpd.GeoDataFrame(pd.DataFrame(ids,columns=['id']), geometry=geometries_line,crs={'init': 'epsg:26917'})
            gdf.to_file(datafile.replace('.csv','')+"_shapes_"+str(maxdistance)+"_"+str(minneighbors)+'.shp') 
            gdf_line.to_file(datafile.replace('.csv','')+"_shapes_linestring_"+str(maxdistance)+"_"+str(minneighbors)+'.shp')

